In [7]:
import pandas as pd
from ipynb.fs.defs.webscraping import Paths
from datetime import datetime

In [17]:
df = pd.read_csv(
    fr'{Paths.data_interim}\job_offers_{datetime.today().strftime("%d-%m-%Y")}.csv',
    sep=';',
)
df.head()

,name,company,technology,job,location_city,location_country,salary_low,salary_high,salary_currency
0,Data Analyst,FLYR Poland sp.z.o.o,SQL,data analyst,Kraków,POL,11 000,16 000,PLN
1,Junior Data Analyst,Coinfirm,mysql,data analyst,Zdalna,NaN,3 000,5 000,PLN
2,Junior Technical Product Manager,Scalaric,NaN,data analyst,Kraków,POL,5 000,10 000,PLN
3,Business System Analyst (Data&Analytics),Elitmind,NaN,data analyst,Zdalna,NaN,15 000,21 000,PLN
4,Data analyst (analityk danych),Alterdata.io sp. z o.o.,NaN,data analyst,Zdalna,NaN,12 000,21 000,PLN


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              146 non-null    object
 1   company           146 non-null    object
 2   technology        85 non-null     object
 3   job               146 non-null    object
 4   location_city     146 non-null    object
 5   location_country  41 non-null     object
 6   salary_low        146 non-null    object
 7   salary_high       139 non-null    object
 8   salary_currency   139 non-null    object
dtypes: object(9)
memory usage: 10.4+ KB


### Preparing the data for an analysis

In [19]:
df = df[df['salary_currency'].notna()]

In [20]:
df.loc[df['location_country'].isnull(), 'location_country'] = 'N/A'
df

,name,company,technology,job,location_city,location_country,salary_low,salary_high,salary_currency
0,Data Analyst,FLYR Poland sp.z.o.o,SQL,data analyst,Kraków,POL,11 000,16 000,PLN
1,Junior Data Analyst,Coinfirm,mysql,data analyst,Zdalna,N/A,3 000,5 000,PLN
2,Junior Technical Product Manager,Scalaric,NaN,data analyst,Kraków,POL,5 000,10 000,PLN
3,Business System Analyst (Data&Analytics),Elitmind,NaN,data analyst,Zdalna,N/A,15 000,21 000,PLN
4,Data analyst (analityk danych),Alterdata.io sp. z o.o.,NaN,data analyst,Zdalna,N/A,12 000,21 000,PLN
...,...,...,...,...,...,...,...,...,...
141,Data Scientist,Simon - Kucher & Partners,NaN,data scientist,Zdalna,N/A,13 000,15 000,PLN
142,Senior Data Scientist,Relayr,python,data scientist,Katowice,POL\n + 4,21 000,24 000,PLN
143,Senior Data Scientist,hubQuest,python,data scientist,Zdalna,N/A,25 000,34 000,PLN
144,Data Scientist,Avanade Poland,python,data scientist,Warszawa,POL\n + 3,15 000,25 000,PLN


In [22]:
df['location_country'].unique()

array(['POL', 'N/A', 'HUN\n + 1', 'POL\n + 4', 'POL\n + 3', 'POL\n + 1',
       'HUN'], dtype=object)

In [23]:
df.loc[
    (df['location_country'] != 'POL') &
    (df['location_country'] != 'HUN') &
    (df['location_country'] != r'N/A'),
    'location_country'
     ] = 'POL'
df['location_country'].unique()

array(['POL', 'N/A', 'HUN'], dtype=object)

**Converting salary columns to numeric values and adding a column containing the average salary**

In [24]:
dfcopy = df.copy()
dfcopy['salary_low'] = pd.to_numeric(dfcopy['salary_low'].str.replace(" ",""))
dfcopy['salary_high'] = pd.to_numeric(dfcopy['salary_high'].str.replace(" ",""))

In [25]:
dfcopy['salary_average'] = dfcopy[['salary_low', 'salary_high']].mean(axis=1)

In [26]:
df['salary_average'] = dfcopy['salary_average']

**Removing alternative names of the same city and adding a column determining wether a job offer is for a senior title**

In [27]:
df['location_city'].unique()

array(['Kraków', 'Zdalna', 'Gdynia', 'Warszawa', 'Warsaw', 'Budapest',
       'Gdańsk', 'Wrocław', 'Poznań', 'Malbork', 'Łódź', 'Katowice'],
      dtype=object)

In [28]:
df.loc[df['location_city'] == 'Warsaw', 'location_city'] = 'Warszawa'
df.loc[df['name'].str.contains("Senior", case=False), 'is_senior'] = True
df.loc[df['is_senior'].isnull(), 'is_senior'] = False

**Saving the prepared data**

In [29]:
df.to_csv(
    fr'{Paths.data_processed}\job_offers_{datetime.today().strftime("%d-%m-%Y")}.csv',
    sep=';',
    encoding='utf-8',
    index=False
)